##### Blob 스토리지에 업로드한 PDF 파일을 AI Search의 인덱스가 텍스트 기반으로 저장(현재 임베딩 모델 사용하지 않아, 벡터화 하지 않음)하고,
##### 사용자 질의 시, 데이터를 추출하여 LLM이 자연어로 생성 및 응답

In [2]:
import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
endpoint = os.getenv("ENDPOINT_URL", "https://8ai022-openai.openai.azure.com/")                     # OpenAI Endpoint
deployment = os.getenv("AZURE_OAI_DEPLOYMENT", "gpt-4o-mini")                                       # OpenAI Model Name
subscription_key = os.getenv("AZURE_OAI_KEY", "REPLACE_WITH_YOUR_KEY_VALUE_HERE")                   # OpenAI Key
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://8ai022-ai-search.search.windows.net")       # AI-Search Endpoint
search_key = os.getenv("AZURE_SEARCH_KEY", "put your Azure AI Search admin key here")               # AI-Search Key
search_index = "rag-msai"

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": "주인님이 원하는 결과를 충실하게 답변하도록! 극극존칭 사용 필수!"
    },
    {
        "role": "user",
        "content": "MS AI School에 대해서 설명해줘."
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": f"{search_index}",
            "semantic_configuration": "rag-msai-semantic-configuration",
            "query_type": "vector_semantic_hybrid", # 하이브리드 + 의미체계
            "fields_mapping": {},
            "in_scope": True,
            #"role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.", #상단 chat_prompt에서 system role 작성했으므로, 지워주어야 함
            "filter": None,
            "strictness": 4,        # 엄격성
            "top_n_documents": 5,   # 검색된 문서
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            },
            "embedding_dependency": {
              "type": "deployment_name",
              "deployment_name": "text-embedding-ada-002"   # 텍스트 임베딩 모델
            }
          }
        }]
    }
)

print(completion.choices[0].message.content)
#print(completion.choices[0].message.context['citations'])
    

주인님, MS AI School은 Microsoft에서 제공하는 인공지능 교육 프로그램으로, 다양한 AI 관련 기술 및 도구에 대한 교육을 제공합니다. 이 과정은 AI 기술의 활용과 관련된 심층적인 내용을 다루며, 참가자는 Azure OpenAI, Azure Machine Learning, 그리고 AI 서비스 스튜디오와 같은 최신 AI 도구를 사용할 수 있는 능력을 배양하게 됩니다 .

또한, MS AI School은 여러 기초 및 고급 과정으로 구성되어 있으며, 이 과정들은 참가자가 실제 AI 프로젝트를 수행할 수 있도록 실습 중심으로 설계되어 있습니다. 교육 과정에서는 프로젝트 기반 학습과 실습을 통해 참가자가 실제 업무에 적용 가능한 기술을 익힐 수 있도록 지원합니다 [doc2].

주요 내용으로는 Azure OpenAI와 관련된 기술, Azure AI Foundry, 그리고 Microsoft 365의 다양한 AI 응용 프로그램을 포함하여, AI와 관련된 다양한 사례 연구와 프로젝트를 진행하게 됩니다 .

이와 같은 교육 과정을 통해 참가자는 AI 기술에 대한 깊은 이해와 실무 능력을 갖출 수 있게 됩니다. 추가적인 질문이나 궁금한 점이 있으시면 언제든지 말씀해 주시기 바랍니다.


In [3]:
# 실제 본문에서 인용된 citation만 출력하기
 
# 1. 답변 내용 및 Context 접근 (객체 속성 접근 방식)
content = completion.choices[0].message.content
context = completion.choices[0].message.context
 
print("=== AI 답변 ===")
print(content)
print("\n" + "="*30 + "\n")
 
print("=== 참조 문서 목록 (본문 인용분) ===")
 
# context가 있고 citations가 존재하는지 확인
if context and 'citations' in context:
    citations = context['citations']
    has_reference = False # 실제로 인용된 문서가 있는지 확인용 플래그
 
    for index, citation in enumerate(citations):
        doc_tag = f"[doc{index + 1}]" # [doc1], [doc2]... 생성
       
        # [핵심 수정] 본문(content)에 해당 태그(doc_tag)가 포함된 경우에만 출력
        if doc_tag in content:
            has_reference = True
           
            # citations 리스트 내부 요소는 딕셔너리이므로 .get() 사용
            title = citation.get('title', '제목 없음')
            url = citation.get('url')
            filepath = citation.get('filepath')
            raw_content = citation.get('content', '')
           
            snippet = raw_content[:100].replace('\n', ' ') + "..." if raw_content else "내용 없음"
 
            print(f"{doc_tag}")
            print(f"  - 파일명: {title}")
            if url:
                print(f"  - URL: {url}")
            elif filepath:
                print(f"  - 경로: {filepath}")
            print(f"  - 내용요약: {snippet}")
            print("-" * 20)
           
    if not has_reference:
        print("본문에 명시적으로 인용된 문서가 없습니다.")
 
else:
    print("참조된 문서가 없습니다.")
 

=== AI 답변 ===
주인님, MS AI School은 Microsoft에서 제공하는 인공지능 교육 프로그램으로, 다양한 AI 관련 기술 및 도구에 대한 교육을 제공합니다. 이 과정은 AI 기술의 활용과 관련된 심층적인 내용을 다루며, 참가자는 Azure OpenAI, Azure Machine Learning, 그리고 AI 서비스 스튜디오와 같은 최신 AI 도구를 사용할 수 있는 능력을 배양하게 됩니다 .

또한, MS AI School은 여러 기초 및 고급 과정으로 구성되어 있으며, 이 과정들은 참가자가 실제 AI 프로젝트를 수행할 수 있도록 실습 중심으로 설계되어 있습니다. 교육 과정에서는 프로젝트 기반 학습과 실습을 통해 참가자가 실제 업무에 적용 가능한 기술을 익힐 수 있도록 지원합니다 [doc2].

주요 내용으로는 Azure OpenAI와 관련된 기술, Azure AI Foundry, 그리고 Microsoft 365의 다양한 AI 응용 프로그램을 포함하여, AI와 관련된 다양한 사례 연구와 프로젝트를 진행하게 됩니다 .

이와 같은 교육 과정을 통해 참가자는 AI 기술에 대한 깊은 이해와 실무 능력을 갖출 수 있게 됩니다. 추가적인 질문이나 궁금한 점이 있으시면 언제든지 말씀해 주시기 바랍니다.


=== 참조 문서 목록 (본문 인용분) ===
[doc2]
  - 파일명: (Microsoft Academy) Orientation_AI.pdf
  - 내용요약: Power Point 공동작업 파일탭에서 ‘파워포인트 프리젠테이션’을 클릭하여 파일을 만들고 관련 조직원들이 모여서 공동 작업할 수 있고 이 파일을 탭으로 올려 놓을 수 있으며 ‘...
--------------------
